In [14]:
!sudo apt-get update && sudo apt-get install -y espeak
!pip install librosa soundfile transformers torch sequence_align python-dotenv phonemizer
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Tokenizer
import torch
import librosa # Import librosa for audio loading and resampling
import soundfile as sf # Still useful for basic audio file checks, though librosa.load can handle it
from sequence_align.pairwise import needleman_wunsch


# audio_file_path = "/content/achyut_pizza_false.wav"

TARGET_SAMPLE_RATE = 16000
model_name = "facebook/wav2vec2-xlsr-53-espeak-cv-ft"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,268 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,159 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2PhonemeCTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [23]:
# Get the list of phoneme tokens
phoneme_vocab_dict = tokenizer.get_vocab()

# Map: phoneme → ID
special_tokens = [] # ['<s>', '</s>', '<pad>', '|']
phoneme_to_index = {p: i for p, i in tokenizer.get_vocab().items() if p not in special_tokens}

# Map: ID → phoneme (optional, for decoding)
index_to_phoneme = {idx: phoneme for phoneme, idx in phoneme_vocab_dict.items()}

In [24]:
from dotenv import load_dotenv
from phonemizer.punctuation import Punctuation
from phonemizer.backend import EspeakBackend
from phonemizer.separator import Separator
from sequence_align.pairwise import needleman_wunsch


load_dotenv()
from openai import OpenAI

client = OpenAI(api_key='sk-')
def transcribe_audio(path):
    audio_file= open(path, "rb")

    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        prompt="matlab, jaise ki, vagera-vagera, I'm like,you know what I mean, kind of, um, ah, huh, and so, so um, uh, and um, like um, so like, like it's, it's like, i mean, yeah, ok so, uh so, so uh, yeah so, you know, it's uh, uh and, and uh, like, kind",
    )

    return transcription.text

from text_unidecode import unidecode

def generate_reference_phoneme(reference_text):
    # Clean punctuation and lowercase
    text = unidecode(reference_text)  # handles accents safely
    cleaned = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in text)
    ref_words = [w.lower() for w in cleaned.strip().split() if w]

    backend = EspeakBackend('en-us')
    separator = Separator(phone='', word=' ')  # important for word separation

    # Batch phonemize all words
    phonemes = backend.phonemize(ref_words, separator=separator, strip=True)

    # Zip together
    lexicon = list(zip(ref_words, phonemes))

    return lexicon, ref_words

def expected_phonetics(path,transcript=None):
    if not transcript:
        transcript = transcribe_audio(path=path)

    lexicon, ref_words = generate_reference_phoneme(transcript)
    return transcript, lexicon, ref_words

def genarate_recorded_phonetics(audio_file_path:str):
  # 1. Load your audio file and resample to TARGET_SAMPLE_RATE (16kHz)
  try:
      # librosa.load returns the audio time series and the sampling rate
      audio_input, sample_rate = librosa.load(audio_file_path, sr=TARGET_SAMPLE_RATE, mono=True)
      # By setting sr=TARGET_SAMPLE_RATE, librosa automatically resamples if needed
      # By setting mono=True, librosa automatically converts to mono if needed
  except FileNotFoundError:
      print(f"Error: Audio file not found at {audio_file_path}")
      exit()
  except Exception as e:
      print(f"Error loading or processing audio file: {e}")
      exit()

  input_values = processor(audio_input, return_tensors="pt", sampling_rate=TARGET_SAMPLE_RATE).input_values

  # Retrieve logits
  with torch.no_grad():
      logits = model(input_values).logits

  # Take argmax and decode
  predicted_ids = torch.argmax(logits, dim=-1)
  posteriors = torch.nn.functional.softmax(logits, dim=-1)
  transcription = processor.batch_decode(predicted_ids)
  predicted_ids = torch.argmax(logits, dim=-1)  # shape: [batch, time]

  # Filter each sample in batch
  filtered_ids_batch = []
  filtered_posteriors_batch = []

  for i in range(predicted_ids.size(0)):  # iterate over batch
      ids = predicted_ids[i]
      probs = posteriors[i]

      filtered_ids = []
      filtered_probs = []

      prev_id = -1  # initialize to an invalid ID
      for j in range(ids.size(0)):
          curr_id = ids[j].item()
          if curr_id == 0:
              continue  # skip blank (or pad) token
          if curr_id == prev_id:
              continue  # skip consecutive duplicates
          filtered_ids.append(curr_id)
          filtered_probs.append(probs[j])
          prev_id = curr_id

      filtered_ids_batch.append(torch.tensor(filtered_ids))
      filtered_posteriors_batch.append(torch.stack(filtered_probs))

  # Decode (if needed)
  filtered_transcriptions = processor.batch_decode(
      [ids.tolist() for ids in filtered_ids_batch]
  )

  # Step 2: Convert IDs to phoneme symbols using your index-to-phoneme mapping
  decoded_phonemes = [index_to_phoneme[idx.item()] for idx in filtered_ids_batch[0]]

  return decoded_phonemes,filtered_posteriors_batch[0],filtered_transcriptions

In [25]:
def align_phonetics(seq_a,seq_b):

  aligned_seq_a, aligned_seq_b = needleman_wunsch(
      seq_a,
      seq_b,
      match_score=1.0,
      mismatch_score=-1.0,
      indel_score=-1.0,
      gap="_",
  )
  aligned_reference_seq = ''.join(aligned_seq_a)
  aligned_recorded_seq = ''.join(aligned_seq_b)

  return aligned_reference_seq,aligned_recorded_seq

In [26]:
import pandas as pd

In [27]:
data = pd.read_csv('/content/dataset.csv')

In [28]:
def find_and_add_space(ref_phonemes, recorded_str):
    # print("CALLED",ref_phonemes, recorded_str)
    new_str = ''
    for ix,ch in enumerate(list(ref_phonemes)):
      if ch == ' ':
        new_str += ' '
      else:
        new_str += recorded_str[ix]
    return new_str

In [29]:
import torch
import math

def compute_gop_scores(posteriors, aligned_reference_list,aligned_recorded_list, threshold=-0.5):
    """
    Computes GOP scores for each time-aligned phoneme.

    Args:
        posteriors (torch.Tensor): Softmax probabilities of shape [T, V]
        aligned_reference_list (List[str]): Ground-truth aligned phonemes (length T)
        phoneme_to_index (Dict[str, int]): Mapping from phoneme symbol to vocabulary index
        threshold (float): GOP threshold to flag mispronunciations (default: -0.5)

    Returns:
        gop_results (List[Tuple[int, str, float, bool]]): List of (t, phoneme, gop_score, is_flagged)
    """
    assert len(aligned_reference_list) == len(aligned_recorded_list), f"Length mismatch: aligned_reference_list ({len(aligned_reference_list)}), posteriors time steps ({posteriors.shape[0]})"

    gop_pred_true = []
    gop_pred_false = []

    gop_results = []
    p_index = 0

    time_steps = len(aligned_reference_list)
    for t in range(time_steps):

        ref_phoneme = aligned_reference_list[t]
        rec_phoneme = aligned_recorded_list[t]
        pred = True if ref_phoneme == rec_phoneme else False

        if ref_phoneme not in phoneme_to_index:
            if ref_phoneme == ' ' or ref_phoneme == 'ː' or reference_phoneme == '_':
                continue

            print(f"[Warning] Unknown phoneme at t={t}: '{ref_phoneme}' → Skipped")
            continue

        idx_ref = phoneme_to_index[ref_phoneme]
        P_ref = posteriors[p_index, idx_ref].item()
        P_max = torch.max(posteriors[p_index]).item()

        if P_ref == 0 or P_max == 0:
            gop = float('-inf')  # To avoid log(0)
        else:
            gop = math.log(P_ref) - math.log(P_max)

        if pred:
            gop_pred_true.append(gop)
        else:
            gop_pred_false.append(gop)

        is_flagged = gop < threshold
        gop_results.append((t, ref_phoneme, gop, is_flagged))

        p_index += 1

    return gop_results,gop_pred_true,gop_pred_false

In [50]:
base_directory = '/content/'
reference_phoneme = ''
phonetics = ''
posteriors = ''
lexicon, ref_words = None,None

def fun():
  for ix,i in data.iterrows():
    print(i['case'])

    path = base_directory + i['case']
    transcript = i['transcript']
    transcript,lexicon, ref_words = expected_phonetics(path=path,transcript=transcript)
    # Clean punctuation and lowercase
    transcript = unidecode(transcript)  # handles accents safely
    transcript = ''.join(c if c.isalnum() or c.isspace() else ' ' for c in transcript)

    reference_phoneme =' '.join([phon for w, phon in lexicon])
    # phonetics,posteriors,_ = genarate_recorded_phonetics(path)
    return genarate_recorded_phonetics(path), lexicon
    print("len rec phonetics list",len(phonetics))
    print("len of posteriors ",posteriors.shape[0])
    phonetics = ''.join(phonetics)
    print("ref ,rec")
    print(reference_phoneme)
    print(phonetics)
    print("len of ref phonetics",len(reference_phoneme),"len of rec phonetics str",len(phonetics))


    aligned_reference_seq,aligned_recorded_seq = align_phonetics(reference_phoneme,phonetics)
    print("aligned rec, aligned ref")
    print(aligned_reference_seq)
    print(aligned_recorded_seq)
    print(len(aligned_reference_seq),len(aligned_recorded_seq))


    spaced_recorded_seq = find_and_add_space(aligned_reference_seq,aligned_recorded_seq)

    print("spaced rec, aligned ref")
    print(spaced_recorded_seq)
    print(aligned_reference_seq)
    print(len(spaced_recorded_seq),len(aligned_reference_seq))

    aligned_recorded_list = list(spaced_recorded_seq)
    aligned_reference_list = list(aligned_reference_seq)

    gop_results,gop_pred_true,gop_pred_false = compute_gop_scores(posteriors,aligned_reference_list,aligned_recorded_list)

    print(gop_results)

    num_flagged = sum(1 for _, _, _, f in gop_results if f)
    print(f"\n🔎 Total flagged phonemes: {num_flagged}/{len(gop_results)}")

In [51]:
(rec_tokens, posteriors, batch_decoded), ref_tokens = fun()aligned_rec_tokens

smit_pizza_false.wav


In [52]:
rec = "".join(rec_tokens)
ref = "".join([j for i,j in ref_tokens])

In [53]:
print(ref)
print(rec)

aɪlʌviːɾɪŋpiːtsəwaɪlwɑːtʃɪŋdɑːkjuːmɛntɚɹizɐbaʊtɹʊɹɹəlɪndɪə
aɪloːuːtɪŋbizɐwaɪlwɔtʃɪŋdɔkiməntidʒɐbɔtɡɹuːɾəlɪndiɐ


In [54]:
aligned_reference_seq,aligned_recorded_seq = align_phonetics(ref,rec)

In [55]:
print(aligned_reference_seq)
print(aligned_recorded_seq)

aɪlʌviːɾɪŋpiːtsəwaɪlwɑːtʃɪŋdɑːkjuːmɛntɚɹi_zɐbaʊt_ɹʊɹɹəlɪndɪə
aɪloːuːtɪŋbi__zɐwaɪlw_ɔtʃɪŋd_ɔk__imənt__idʒɐb_ɔtɡɹuːɾəlɪndiɐ


In [56]:
def tokenise_aligned_rec_seq(rec_tokens,seq):
  tokens = []
  for i in rec_tokens:
    while(seq[0]=='_'):
      tokens.append('_')
      seq = seq[1:]
    if i == seq[:len(i)]:
      tokens.append(i)
      seq = seq[len(i):]
  return tokens

In [57]:
aligned_rec_tokens = tokenise_aligned_rec_seq(rec_tokens,aligned_recorded_seq)
# ['aɪ', 'l', 'oː', 'uː', 't', 'ɪ', 'ŋ', 'b', 'i', '_', '_', 'z', 'ɐ', 'w', 'aɪ', 'l', 'w', '_', 'ɔ', 'tʃ', 'ɪ', 'ŋ', 'd', '_', 'ɔ', 'k', '_', '_', 'i', 'm', 'ə', 'n', 't', '_', '_', 'i', 'dʒ', 'ɐ', 'b', '_', 'ɔ', 't', 'ɡ', 'ɹ', 'uː', 'ɾ', 'ə', 'l', 'ɪ', 'n', 'd', 'i', 'ɐ']

In [39]:
print(rec_tokens)
# ['aɪ', 'l', 'oː', 'uː', 't', 'ɪ', 'ŋ', 'b', 'i', 'z', 'ɐ', 'w', 'aɪ', 'l', 'w', 'ɔ', 'tʃ', 'ɪ', 'ŋ', 'd', 'ɔ', 'k', 'i', 'm', 'ə', 'n', 't', 'i', 'dʒ', 'ɐ', 'b', 'ɔ', 't', 'ɡ', 'ɹ', 'uː', 'ɾ', 'ə', 'l', 'ɪ', 'n', 'd', 'i', 'ɐ']

['aɪ', 'l', 'oː', 'uː', 't', 'ɪ', 'ŋ', 'b', 'i', 'z', 'ɐ', 'w', 'aɪ', 'l', 'w', 'ɔ', 'tʃ', 'ɪ', 'ŋ', 'd', 'ɔ', 'k', 'i', 'm', 'ə', 'n', 't', 'i', 'dʒ', 'ɐ', 'b', 'ɔ', 't', 'ɡ', 'ɹ', 'uː', 'ɾ', 'ə', 'l', 'ɪ', 'n', 'd', 'i', 'ɐ']


In [58]:
print(aligned_reference_seq)
# aɪlʌviːɾɪŋpiːtsəwaɪlwɑːtʃɪŋdɑːkjuːmɛntɚɹi_zɐbaʊt_ɹʊɹɹəlɪndɪə

aɪlʌviːɾɪŋpiːtsəwaɪlwɑːtʃɪŋdɑːkjuːmɛntɚɹi_zɐbaʊt_ɹʊɹɹəlɪndɪə


In [59]:
vocab = list(phoneme_to_index.keys())
# ['<s>', '<pad>', '</s>', '<unk>', 'n', 's', 't', 'ə', 'l', 'a', 'i', 'k', 'd', 'm', 'ɛ', 'ɾ', 'e', 'ɪ', 'p', 'o', 'ɐ', 'z', 'ð', 'f', 'j', 'v', 'b', 'ɹ', 'ʁ', 'ʊ', 'iː', 'r', 'w', 'ʌ', 'u', 'ɡ', 'æ', 'aɪ', 'ʃ', 'h', 'ɔ', 'ɑː', 'ŋ', 'ɚ', 'eɪ', 'β', 'uː', 'y', 'ɑ̃', 'oʊ', 'ᵻ', 'eː', 'θ', 'aʊ', 'ts', 'oː', 'ɔ̃', 'ɣ', 'ɜ', 'ɑ', 'dʒ', 'əl', 'x', 'ɜː', 'ç', 'ʒ', 'tʃ', 'ɔː', 'ɑːɹ', 'ɛ̃', 'ʎ', 'ɔːɹ', 'ʋ', 'aː', 'ɕ', 'œ', 'ø', 'oːɹ', 'ɲ', 'yː', 'ʔ', 'iə', 'i5', 's.', 'tɕ', '??', 'nʲ', 'ɛː', 'œ̃', 'ɭ', 'ɔø', 'ʑ', 'tʲ', 'ɨ', 'ɛɹ', 'ts.', 'rʲ', 'ɪɹ', 'ɭʲ', 'i.5', 'ɔɪ', 'q', 'sʲ', 'u5', 'ʊɹ', 'iɜ', 'a5', 'iɛ5', 'øː', 'ʕ', 'ja', 'əɜ', 'th', 'ɑ5', 'oɪ', 'dʲ', 'ə5', 'tɕh', 'ts.h', 'mʲ', 'ɯ', 'dʑ', 'vʲ', 'e̞', 'tʃʲ', 'ei5', 'o5', 'onɡ5', 'ɑu5', 'iɑ5', 'ai5', 'aɪɚ', 'kh', 'ə1', 'ʐ', 'i2', 'ʉ', 'ħ', 't[', 'aɪə', 'ʲ', 'ju', 'ə2', 'u2', 'oɜ', 'pː', 'iɛɜ', 'ou5', 'y5', 'uɜ', 'tː', 'uo5', 'd[', 'uoɜ', 'tsh', 'ɑɜ', 'ɵ', 'i̪5', 'uei5', 'ɟ', 'aɜ', 'ɑɨ', 'i.ɜ', 'eʊ', 'o2', 'ɐ̃', 'ä', 'pʲ', 'kʲ', 'n̩', 'ɒ', 'ph', 'ɑu2', 'uɨ', 'əɪ', 'ɫ', 'ɬ', 'yɜ', 'bʲ', 'ɑ2', 's̪', 'aiɜ', 'χ', 'ɐ̃ʊ̃', '1', 'ə4', 'yæɜ', 'a2', 'ɨː', 't̪', 'iouɜ', 'ũ', 'onɡɜ', 'aɨ', 'iɛ2', 'ɔɨ', 'ɑuɜ', 'o̞', 'ei2', 'iou2', 'c', 'kː', 'y2', 'ɖ', 'oe', 'dˤ', 'yɛɜ', 'əʊ', 'S', 'ɡʲ', 'onɡ2', 'u"', 'eiɜ', 'ʈ', 'ɯᵝ', 'iou5', 'dZ', 'r̝̊', 'i.2', 'tS', 's^', 'ʝ', 'yə5', 'iɑɜ', 'uə5', 'pf', 'ɨu', 'iɑ2', 'ou2', 'ər2', 'fʲ', 'ai2', 'r̝', 'uəɜ', 'ɳ', 'əɨ', 'ua5', 'uɪ', 'ɽ', 'bː', 'yu5', 'uo2', 'yɛ5', 'l̩', 'ɻ', 'ərɜ', 'ʂ', 'i̪2', 'ouɜ', 'uaɜ', 'a.', 'a.ː', 'yæ5', 'dː', 'r̩', 'ee', 'ɪu', 'ər5', 'i̪ɜ', 'æi', 'u:', 'i.ː', 't^', 'o1', 'ɪ^', 'ai', 'ueiɜ', 'æː', 'ɛɪ', 'eə', 'i.', 'ɴ', 'ie', 'ua2', 'ɑ1', 'o4', 'tʃː', 'o:', 'ɑ:', 'u1', 'N', 'i̪1', 'au', 'yæ2', 'u.', 'qː', 'yəɜ', 'y:', 'kʰ', 'tʃʰ', 'iʊ', 'sx', 'õ', 'uo', 'tʰ', 'uai5', 'bʰ', 'u.ː', 'uə2', 'ʊə', 'd^', 's̪ː', 'yiɜ', 'dʰ', 'r.', 'oe:', 'i1', 'ɟː', 'yu2', 'nʲʲ', 'i̪4', 'uei2', 'tsʲ', 'ɸ', 'ĩ', 'ɑ4', 't̪ː', 'eɑ', 'u4', 'e:', 'tsː', 'ʈʰ', 'ɡʰ', 'ɯɯ', 'dʒʲ', 'ʂʲ', 'X', 'ɵː', 'uaiɜ', 'tɕʲ', 'ã', 't^ː', 'ẽː', 'yɛ2', 'cː', 'i.1', 'ɛʊ', 'dˤdˤ', 'dʒː', 'i4', 'ɡː', 'yi', 'ɕʲ', 'ɟʰ', 'pʰ', 'dʑʲ', 'yuɜ', 'ua1', 'ua4', 'æiː', 'ɐɐ', 'ui', 'iou1', 'ʊː', 'a1', 'iou4', 'cʰ', 'iɛ1', 'yə2', 'ɖʰ', 'ẽ', 'ʒʲ', 'ää', 'ər4', 'iːː', 'ɪː', 'iɑ1', 'ər1', 'œː', 'øi', 'ɪuː', 'cʰcʰ', 'əː1', 'iː1', 'ũ', 'kʰː', 'o̞o̞', 'xʲ', 'ou1', 'iɛ4', 'e̞e̞', 'y1', 'dzː', 'dʲʲ', 'dʰː', 'ɯᵝɯᵝ', 'lː', 'uo1', 'i.4', 'i:', 'yɛ5ʲ', 'a4']

In [68]:
# Gemini 2.5 pro

import collections

def tokenize_aligned_sequence(rec_tokens, aligned_reference_seq, vocab):
    """
    Tokenizes the aligned_reference_seq into a list of tokens with the same
    length as rec_tokens, where each token is present in the vocab.

    This function uses dynamic programming to find a valid segmentation of the
    aligned_reference_seq string.

    Args:
        rec_tokens (list): The list of recognized tokens. Its length determines
                           the target number of tokens for the output.
        aligned_reference_seq (str): The reference string to be tokenized.
        vocab (list): A list of valid tokens.

    Returns:
        list: A list of strings representing the tokenized aligned_reference_seq.
              Returns an error message if no valid tokenization is found.
    """
    n = len(aligned_reference_seq)
    k = len(rec_tokens)
    vocab_set = set(vocab)

    # dp[i][j] will be true if the prefix of aligned_reference_seq of length i
    # can be segmented into j tokens from the vocab.
    dp = collections.defaultdict(bool)
    # path[i][j] stores the length of the last token in a valid segmentation
    # of the prefix of length i into j tokens.
    path = collections.defaultdict(int)

    # Base case: an empty string can be tokenized into 0 tokens.
    dp[0, 0] = True

    for j in range(1, k + 1):  # Target number of tokens
        for i in range(1, n + 1):  # Length of the string prefix
            for p in range(i):  # Potential split point
                # Check if the prefix of length p can be formed by j-1 tokens
                if dp[p, j - 1]:
                    last_token = aligned_reference_seq[p:i]
                    # If the last segment is a valid token
                    if last_token in vocab_set:
                        dp[i, j] = True
                        path[i, j] = i - p  # Store the length of this token
                        break # Found a valid segmentation, move to the next prefix length

    # Reconstruct the tokens by backtracking if a solution exists
    if not dp[n, k]:
        return ["Error: A valid tokenization could not be found."]

    result_tokens = []
    curr_n, curr_k = n, k
    while curr_n > 0 and curr_k > 0:
        token_len = path[curr_n, curr_k]
        token = aligned_reference_seq[curr_n - token_len:curr_n]
        result_tokens.append(token)
        curr_n -= token_len
        curr_k -= 1

    # The tokens were added in reverse order, so we reverse them back.
    return result_tokens[::-1]



# Generate the tokenized sequence
aligned_tokens = tokenize_aligned_sequence(aligned_rec_tokens, aligned_reference_seq, list(phoneme_to_index.keys()))

# Print the final result
print(aligned_tokens)

['Error: A valid tokenization could not be found.']


In [69]:
# Deepseek R1

def tokenize_ref(rec_tokens, ref_str, vocab):
    N = len(rec_tokens)
    S = ref_str
    L = len(S)

    if L == 0:
        return [] if N == 0 else None

    vocab_set = set(vocab)
    max_len = max(len(token) for token in vocab) if vocab else 0

    dp = [[False] * (L + 1) for _ in range(N + 1)]
    parent = [[-1] * (L + 1) for _ in range(N + 1)]

    dp[0][0] = True

    for i in range(1, N + 1):
        for j in range(i, L + 1):
            start_index = max(0, j - max_len)
            for k in range(start_index, j):
                token = S[k:j]
                if token in vocab_set and dp[i - 1][k]:
                    dp[i][j] = True
                    parent[i][j] = k
                    break

    if not dp[N][L]:
        return None

    tokens = []
    j = L
    for i in range(N, 0, -1):
        k = parent[i][j]
        tokens.append(S[k:j])
        j = k
    tokens.reverse()
    return tokens

result = tokenize_ref(aligned_rec_tokens, aligned_reference_seq, vocab)
print(result)

None


In [71]:
# Claude Sonnet 4

def tokenize_aligned_sequence(aligned_reference_seq, rec_tokens, vocab):
    """
    Tokenize aligned_reference_seq using vocab while maintaining alignment with rec_tokens.

    Args:
        aligned_reference_seq: String of phonemes to tokenize
        rec_tokens: List of recognized tokens for alignment
        vocab: List of vocabulary tokens

    Returns:
        List of tokens from vocab that align with rec_tokens
    """

    # Create a set for faster vocab lookup
    vocab_set = set(vocab)

    # Remove special tokens for the main processing
    special_tokens = {'<s>', '</s>', '<unk>'}
    vocab_phonemes = [token for token in vocab if token not in special_tokens]

    # Dynamic programming approach to find optimal segmentation
    def find_best_segmentation(sequence, target_length):
        n = len(sequence)
        if n == 0:
            return []

        # dp[i][j] = (cost, segmentation) for segmenting sequence[:i] into j tokens
        # cost is based on how well the segmentation matches the target length
        dp = {}

        def solve(pos, tokens_used):
            if pos == n:
                # Penalize if we don't match target length exactly
                length_penalty = abs(tokens_used - target_length) * 10
                return (length_penalty, [])

            if (pos, tokens_used) in dp:
                return dp[(pos, tokens_used)]

            best_cost = float('inf')
            best_segmentation = []

            # Try all possible tokens starting at current position
            for token in vocab_phonemes:
                if sequence[pos:].startswith(token):
                    token_len = len(token)
                    remaining_pos = pos + token_len

                    # Recursively solve for the rest
                    rest_cost, rest_segmentation = solve(remaining_pos, tokens_used + 1)

                    # Cost is based on length penalty and preference for longer matches
                    token_cost = 1.0 / len(token)  # Prefer longer tokens
                    total_cost = rest_cost + token_cost

                    if total_cost < best_cost:
                        best_cost = total_cost
                        best_segmentation = [token] + rest_segmentation

            # If no valid token found, use <unk>
            if best_cost == float('inf'):
                if pos + 1 <= n:
                    rest_cost, rest_segmentation = solve(pos + 1, tokens_used + 1)
                    best_cost = rest_cost + 100  # High penalty for unknown
                    best_segmentation = ['<unk>'] + rest_segmentation
                else:
                    best_cost = abs(tokens_used - target_length) * 10
                    best_segmentation = []

            dp[(pos, tokens_used)] = (best_cost, best_segmentation)
            return best_cost, best_segmentation

        _, segmentation = solve(0, 0)
        return segmentation

    # Get the target length from rec_tokens
    target_length = len(rec_tokens)

    # Find the best segmentation
    tokenized_sequence = find_best_segmentation(aligned_reference_seq, target_length)

    # If the length doesn't match exactly, try to adjust
    while len(tokenized_sequence) < target_length:
        # Add padding tokens or split existing tokens if possible
        tokenized_sequence.append('_')

    while len(tokenized_sequence) > target_length:
        # Try to merge some tokens or remove padding
        if '_' in tokenized_sequence:
            tokenized_sequence.remove('_')
        else:
            tokenized_sequence.pop()

    return tokenized_sequence

# Alternative greedy approach that might work better for your specific case
def tokenize_aligned_sequence_greedy(aligned_reference_seq, rec_tokens, vocab):
    """
    Greedy tokenization approach that prioritizes matching the target length.
    """
    vocab_set = set(vocab)
    special_tokens = {'<s>', '</s>', '<unk>'}
    vocab_phonemes = sorted([token for token in vocab if token not in special_tokens],
                           key=len, reverse=True)  # Prefer longer tokens

    target_length = len(rec_tokens)
    sequence = aligned_reference_seq
    result = []
    pos = 0

    while pos < len(sequence) and len(result) < target_length:
        found_token = False

        # Calculate how much sequence is left and how many tokens we need
        remaining_sequence = len(sequence) - pos
        remaining_tokens_needed = target_length - len(result)

        if remaining_tokens_needed <= 0:
            break

        # Prefer tokens that give us a good length ratio
        target_token_length = remaining_sequence / remaining_tokens_needed

        best_token = None
        best_score = float('inf')

        for token in vocab_phonemes:
            if sequence[pos:].startswith(token):
                # Score based on how close the token length is to our target
                length_diff = abs(len(token) - target_token_length)
                score = length_diff

                if score < best_score:
                    best_score = score
                    best_token = token

        if best_token:
            result.append(best_token)
            pos += len(best_token)
            found_token = True

        if not found_token:
            # Skip one character and add unknown token
            result.append('<unk>')
            pos += 1

    # Pad or trim to match target length
    while len(result) < target_length:
        result.append('_')

    result = result[:target_length]

    return result


# Test both approaches
print("Testing greedy approach:")
result_greedy = tokenize_aligned_sequence_greedy(aligned_reference_seq, aligned_rec_tokens, vocab)
print(f"Length of result: {len(result_greedy)}")
print(f"Length of rec_tokens: {len(aligned_rec_tokens)}")
print(f"Result: {result_greedy}")

print("\nTesting dynamic programming approach:")
result_dp = tokenize_aligned_sequence(aligned_reference_seq, aligned_rec_tokens, vocab)
print(f"Length of result: {len(result_dp)}")
print(f"Length of rec_tokens: {len(aligned_rec_tokens)}")
print(f"Result: {result_dp}")

# Verify all tokens are in vocab
print(f"\nAll tokens in vocab (greedy): {all(token in vocab for token in result_greedy)}")
print(f"All tokens in vocab (DP): {all(token in vocab for token in result_dp)}")

# Show alignment comparison
print(f"\nFirst 10 tokens comparison:")
print(f"rec_tokens:    {aligned_rec_tokens[:10]}")
print(f"result_greedy: {result_greedy[:10]}")
print(f"result_dp:     {result_dp[:10]}")

Testing greedy approach:
Length of result: 53
Length of rec_tokens: 53
Result: ['a', 'ɪ', 'l', 'ʌ', 'v', 'i', '<unk>', 'ɾ', 'ɪ', 'ŋ', 'p', 'i', '<unk>', 't', 's', 'ə', 'w', 'a', 'ɪ', 'l', 'w', 'ɑ', '<unk>', 't', 'ʃ', 'ɪ', 'ŋ', 'd', 'ɑ', '<unk>', 'k', 'j', 'u', '<unk>', 'm', 'ɛ', 'n', 't', 'ɚ', 'ɹ', 'i', '<unk>', 'z', 'ɐ', 'b', 'aʊ', 't', '<unk>', 'ɹ', 'ʊɹ', 'ɹ', 'əl', 'ɪ']

Testing dynamic programming approach:
Length of result: 53
Length of rec_tokens: 53
Result: ['a', 'ɪ', 'l', 'ʌ', 'v', 'iː', 'ɾ', 'ɪ', 'ŋ', 'p', 'iː', 't', 's', 'ə', 'w', 'a', 'ɪ', 'l', 'w', 'ɑː', 't', 'ʃ', 'ɪ', 'ŋ', 'd', 'ɑː', 'k', 'j', 'uː', 'm', 'ɛ', 'n', 't', 'ɚ', 'ɹ', 'i', '<unk>', 'z', 'ɐ', 'b', 'a', 'ʊ', 't', '<unk>', 'ɹ', 'ʊɹ', 'ɹ', 'əl', 'ɪ', 'n', 'd', 'ɪ', 'ə']

All tokens in vocab (greedy): True
All tokens in vocab (DP): True

First 10 tokens comparison:
rec_tokens:    ['aɪ', 'l', 'oː', 'uː', 't', 'ɪ', 'ŋ', 'b', 'i', '_']
result_greedy: ['a', 'ɪ', 'l', 'ʌ', 'v', 'i', '<unk>', 'ɾ', 'ɪ', 'ŋ']
result_dp:     

In [72]:
print(aligned_rec_tokens)
print(result_dp)

['aɪ', 'l', 'oː', 'uː', 't', 'ɪ', 'ŋ', 'b', 'i', '_', '_', 'z', 'ɐ', 'w', 'aɪ', 'l', 'w', '_', 'ɔ', 'tʃ', 'ɪ', 'ŋ', 'd', '_', 'ɔ', 'k', '_', '_', 'i', 'm', 'ə', 'n', 't', '_', '_', 'i', 'dʒ', 'ɐ', 'b', '_', 'ɔ', 't', 'ɡ', 'ɹ', 'uː', 'ɾ', 'ə', 'l', 'ɪ', 'n', 'd', 'i', 'ɐ']
['a', 'ɪ', 'l', 'ʌ', 'v', 'iː', 'ɾ', 'ɪ', 'ŋ', 'p', 'iː', 't', 's', 'ə', 'w', 'a', 'ɪ', 'l', 'w', 'ɑː', 't', 'ʃ', 'ɪ', 'ŋ', 'd', 'ɑː', 'k', 'j', 'uː', 'm', 'ɛ', 'n', 't', 'ɚ', 'ɹ', 'i', '<unk>', 'z', 'ɐ', 'b', 'a', 'ʊ', 't', '<unk>', 'ɹ', 'ʊɹ', 'ɹ', 'əl', 'ɪ', 'n', 'd', 'ɪ', 'ə']
